<a href="https://www.kaggle.com/code/mmellinger66/ps3e6-paris-housing-models?scriptVersionId=119688924" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

 <div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Playground Season 3: Episode 6 - Paris Housing Prices</h1>
</div>

## Problem Type

Regression

## Evaluation Metric


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Import Libraries</h1>
</div>

In [1]:
from typing import List, Set, Dict, Tuple, Optional

import os
import time
from pathlib import Path
import glob
import gc

import pandas as pd
import numpy as np

from sklearn import impute
from sklearn import metrics
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import svm
from sklearn import cluster
from sklearn import model_selection
from sklearn import ensemble
from sklearn import datasets

import xgboost as xgb
import catboost as cb
import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Visualization Libraries
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
import missingno as msno
from folium import Map
from folium.plugins import HeatMap
from IPython.display import display_html, display_markdown, display_latex
from colorama import Fore, Style

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", 999)
pd.set_option("display.precision", 5)

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Configuration</h1>
</div>

In [2]:
TARGET="price"
ID="id"

In [3]:
class Config:
    path:str = "../input/playground-series-s3e6/"
    gpu:bool = True
    optimize:bool = False
    n_optuna_trials:int = 5
    fast_render:bool = False
    calc_probability:bool = False
    debug:bool = False
    seed:int = 42
    N_ESTIMATORS:int = 2000  # 100, 300, 1000, 2000, 5000, 15_000, 20_000 GBDT
    GPU_N_ESTIMATORS:int = 2000 # Want models to run fast during dev
    N_FOLDS:int = 10

In [4]:
class clr:
    S = Style.BRIGHT + Fore.LIGHTRED_EX
    E = Style.RESET_ALL

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Library</h1>
</div>

In [5]:
def read_data(path: str, analyze:bool=True) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame):
    data_dir = Path(path)

    train = pd.read_csv(data_dir / "train.csv")
    test = pd.read_csv(data_dir / "test.csv")
    submission_df = pd.read_csv(data_dir / "sample_submission.csv")

    if analyze:
        print(clr.S + "=== Shape of Data ==="+clr.E)
        print(f" train data: Rows={train.shape[0]}, Columns={train.shape[1]}")
        print(f" test data : Rows={test.shape[0]}, Columns={test.shape[1]}")

        print(clr.S + "\n=== Train Data: First 5 Rows ===\n"+clr.E)
        display(train.head())
        print(f"\n{clr.S}=== Train Column Names ==={clr.E}\n")
        display(train.columns)
        print(f"\n{clr.S}=== Features/Explanatory Variables ==={clr.E}\n")
        eval_features(train)
        print(f"\n{clr.S}=== Skewness ==={clr.E}\n")
        check_skew(train)
    return train, test, submission_df

def create_submission(model_name: str, target, preds, seed:int=42, nfolds:int=5) -> pd.DataFrame:
    sample_submission[target] = preds #.astype(int)

    if len(model_name) > 0:
        fname = f"submission_{model_name}_k{nfolds}_s{seed}.csv"
    else:
        fname = "submission.csv"

    sample_submission.to_csv(fname, index=False)

    return sample_submission

def show_classification_scores(ground_truth:List[int], yhat:List[int]) -> None:
    accuracy = metrics.accuracy_score(ground_truth, yhat)
    precision = metrics.precision_score(ground_truth, yhat)
    recall = metrics.recall_score(ground_truth, yhat)
    roc = metrics.roc_auc_score(ground_truth, yhat)
    f1 = metrics.f1_score(ground_truth, yhat)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"ROC: {roc:.4f}")
    print(f"f1: {f1:.4f}")
    

def label_encoder(train:pd.DataFrame, test:pd.DataFrame, columns:List[str]) -> (pd.DataFrame, pd.DataFrame) :
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = preprocessing.LabelEncoder().fit_transform(train[col])
        test[col] = preprocessing.LabelEncoder().fit_transform(test[col])
    return train, test   

def create_strat_folds(df:pd.DataFrame, TARGET, n_folds:int=5, seed:int=42) -> pd.DataFrame:
    print(f"TARGET={TARGET}, n_folds={n_folds}, seed={seed}")
    df["fold"] = -1

    kf = model_selection.StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    # kf = GroupKFold(n_splits=Config.N_FOLDS)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(df, df[TARGET])):
        df.loc[valid_idx, "fold"] = fold

    # df.to_csv(f"train_fold{num_folds}.csv", index=False)
    return df


def create_folds(df:pd.DataFrame, n_folds:int=5, seed:int=42) -> pd.DataFrame:
    print(f"n_folds={n_folds}, seed={seed}")
    df["fold"] = -1

    kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=seed)

    for fold, (train_idx, valid_idx) in enumerate(kf.split(df)):
        df.loc[valid_idx, "fold"] = fold

    # df.to_csv(f"train_fold{num_folds}.csv", index=False)
    return df

def show_fold_scores(scores: List[float]) -> (float, float):
    cv_score = np.mean(scores)  # Used in filename
    std_dev = np.std(scores)
    print(
        f"Scores -> Adjusted: {np.mean(scores) - np.std(scores):.8f} , mean: {np.mean(scores):.8f}, std: {np.std(scores):.8f}"
    )
    return cv_score, std_dev


def feature_distribution_types(df:pd.DataFrame, display:bool=True) -> (List[str], List[str]):
    continuous_features = list(df.select_dtypes(include=['int64', 'float64', 'uint8']).columns)
    categorical_features = list(df.select_dtypes(include=['object', 'bool']).columns)
    if display:
        print(f"{clr.S}Continuous Features={continuous_features}{clr.E}\n")
        print(f"{clr.S}Categorical Features={categorical_features}{clr.E}")
    return continuous_features, categorical_features   

def show_cardinality(df:pd.DataFrame, features:List[str]) -> None:
    print("=== Cardinality ===")
    print(df[features].nunique())

## === Model Support ===    

from scipy.stats import mode


def merge_test_predictions(final_test_predictions:List[float], calc_probability:bool=True) -> List[float]:

    if calc_probability:
        print("Mean")
        result = np.mean(np.column_stack(final_test_predictions), axis=1)
    else:
        print("Mode")
        mode_result = mode(np.column_stack(final_test_predictions), axis=1)
        result = mode_result[0].ravel()

    return result

def summary_statistics(X:pd.DataFrame, enhanced=True) -> None:
    desc = X.describe()
    if enhanced:
        desc.loc["var"] = X.var(numeric_only=True).tolist()
        desc.loc["skew"] = X.skew(numeric_only=True).tolist()
        desc.loc["kurt"] = X.kurtosis(numeric_only=True).tolist()

    with pd.option_context("display.precision", 2):
        style = desc.transpose().style.background_gradient(
            cmap="coolwarm"
        )  # .set_precision(4)
    display(style)
    
def show_missing_features(df:pd.DataFrame) -> None:
    missing_vals = df.isna().sum().sort_values(ascending=False)
    print(missing_vals[missing_vals > 0])


def show_duplicate_records(df:pd.DataFrame) -> None:
    dups = df.duplicated()
    print(dups.sum())


def eval_features(df:pd.DataFrame) -> (List[str], List[str], List[str]):
    ## Separate Categorical and Numerical Features
    categorical_features = list(
        df.select_dtypes(include=["category", "object"]).columns
    )
    continuous_features = list(df.select_dtypes(include=["number"]).columns)

    print(f"{clr.S}Continuous features:{clr.E} {continuous_features}")
    print(f"{clr.S}Categorical features:{clr.E} {categorical_features}")
    print("\n --- Cardinality of Categorical Features ---\n")

    for feature in categorical_features:
        cardinality = df[feature].nunique()
        if cardinality < 10:
            print(f"{clr.S}{feature}{clr.E}: cardinality={cardinality}, {df[feature].unique()}")
        else:
            print(f"{clr.S}{feature}{clr.E}: cardinality={cardinality}")
    all_features = categorical_features + continuous_features
    return all_features, categorical_features, continuous_features


def show_feature_importance(feature_importance_lst:List[str]) -> None:
    fis_df = pd.concat(feature_importance_lst, axis=1)

    fis_df.sort_values("0_importance", ascending=True).head(40).plot(
        kind="barh", figsize=(12, 12), title="Feature Importance Across Folds"
    )
    plt.show()


def show_feature_target_crosstab(df:pd.DataFrame, feature_lst:List[str], target:str) -> None:
    for feature in feature_lst:
        print(f"\n=== {feature} vs {target} ===\n")
        display(
            pd.crosstab(df[feature], df[target], margins=True)
        )  # display keeps bold formatting


def show_cardinality(df:pd.DataFrame, features:List[str]) -> None:
    print(f"{clr.S}=== Cardinality ==={clr.E}")
    print(df[features].nunique())


def show_unique_features(df:pd.DataFrame, features:List[str]) -> None:
    for col in features:
        print(col, sorted(df[col].dropna().unique()))


def feature_distribution_types(df:pd.DataFrame, display:bool=True) -> (List[str], List[str]):
    continuous_features = list(
        df.select_dtypes(include=["int64", "float64", "uint8"]).columns
    )
    categorical_features = list(df.select_dtypes(include=["object", "bool"]).columns)
    if display:
        print(f"{clr.S}Continuous Features={clr.E}{continuous_features}\n")
        print(f"{clr.S}Categorical Features={clr.E}{categorical_features}")
    return continuous_features, categorical_features


def describe(X:pd.DataFrame) -> None:
    "Deprecated: Use summary_statistics()"
    desc = X.describe()
    desc.loc['var'] = X.var(numeric_only=True).tolist()
    desc.loc['skew'] = X.skew(numeric_only=True).tolist()
    desc.loc['kurt'] = X.kurtosis(numeric_only=True).tolist()

    with pd.option_context('display.precision', 2):
        style = desc.transpose().style.background_gradient(cmap='coolwarm') #.set_precision(4)
    display(style)
  

def check_skew(df:pd.DataFrame) -> None:
    skew = df.skew(skipna=True,numeric_only=True).sort_values(ascending=False)
    print(skew)
    
def gpu_ify_lgbm(lgbm_dict):
    lgbm_dict["device"] = "gpu"
    lgbm_dict["boosting_type"] = "gbdt"
    lgbm_dict["gpu_platform_id"] = 0
    lgbm_dict["gpu_device_id"] = 0
    return lgbm_dict

def gpu_ify_cb(params):
    params["task_type"] = "GPU"
    return params    


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Optuna Hyperparameter Optimization Library</h1>
</div>

In [6]:
def objective_xgb(trial, X_train, X_valid, y_train, y_valid):

    xgb_params = {
        #         "objective": trial.suggest_categorical("objective", ["multi:softmax"]),
        #         "eval_metric": "mlogloss",
        #         "objective": "multi:softmax",
        "eval_metric": "auc",  # auc, rmse, mae
        "objective": "binary:logistic",
        #         "enable_categorical": trial.suggest_categorical("use_label_encoder", [True]),
        "use_label_encoder": trial.suggest_categorical("use_label_encoder", [False]),
        "n_estimators": trial.suggest_int("n_estimators", 1000, 5000, 100),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 0.25),
        "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
        "max_depth": trial.suggest_int("max_depth", 1, 20),  # 10
        "gamma": trial.suggest_float("gamma", 0, 100, step=0.1),
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        "tree_method": trial.suggest_categorical(
            "tree_method", ["hist"]
        ),  # hist, gpu_hist
#         "predictor": "gpu_predictor",
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 100),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 100),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "n_jobs": trial.suggest_categorical("n_jobs", [4]),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 1e-1, 1e3),
        # "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
    }

    # Model loading and training
    model = xgb.XGBClassifier(**xgb_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=5000,
        verbose=0,
    )

    print(f"Number of boosting rounds: {model.best_iteration}")
    #     oof = model.predict_proba(X_valid)[:, 1] # Probability
    oof = model.predict(X_valid)  # Classification: 0,1

    return metrics.accuracy_score(y_valid, oof)


def objective_lgbm(trial, X_train, X_valid, y_train, y_valid):

    params = {
        "boosting_type": "gbdt",
        # "objective": trial.suggest_categorical("objective", ["mae", "rmse"]),
        #         "objective": trial.suggest_categorical("objective", ["multi:softprob"]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [1_000]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [5000]),
        "n_estimators": trial.suggest_int("n_estimators", 700, 1000),
        "importance_type": "gain",
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1000),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.1, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 300),
        "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 0.25),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "n_jobs": trial.suggest_categorical("n_jobs", [4]),
        #         'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-1, 1e3),
        # "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
    }
    if Config.gpu:
        params["device_type"] = "gpu"

    # Model loading and training
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        # eval_metric="mae",
        callbacks=[
            lgb.log_evaluation(500),
            lgb.early_stopping(500, False, True),
        ],
    )

    #     print(f"Number of boosting rounds: {model.best_iteration}")
    oof = model.predict(X_valid)

    #     return accuracy_score(y_valid, oof)
    return metrics.roc_auc_score(y_valid, oof)

def objective_cb(trial, X_train, X_valid, y_train, y_valid):

    cb_params = {
        "iterations": 10,  # 1000
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.1, 1.0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1, 100),
        "bagging_temperature": trial.suggest_loguniform(
            "bagging_temperature", 0.1, 20.0
        ),
        "random_strength": trial.suggest_float("random_strength", 1.0, 2.0),
        "depth": trial.suggest_int("depth", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 300),
        "use_best_model": True,
        #         "task_type": "GPU",
        "random_seed": 42,
    }

    # Model loading and training
    model = cb.CatBoostClassifier(**cb_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        # eval_metric="accuracy",
        early_stopping_rounds=500,
        verbose=False,
    )

    # print(f"Number of boosting rounds: {model.best_iteration}")
    # oof = model.predict_proba(X_valid)[:, 1]
    oof = model.predict(X_valid)  # Classification

    return metrics.accuracy_score(y_valid, oof)

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Load Train/Test Data and Analyze</h1>
</div>

## Load the following files

 - train.csv - Data used to build our machine learning model
 - test.csv - Data used to build our machine learning model. Does not contain the target variable
 - sample_submission.csv - A file in the proper format to submit test predictions

In [7]:
%%time
train, test, sample_submission = read_data(Config.path, analyze=True)                                

=== Shape of Data ===
 train data: Rows=22730, Columns=18
 test data : Rows=15154, Columns=17

=== Train Data: First 5 Rows ===



,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8



=== Train Column Names ===



Index(['id', 'squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors',
       'cityCode', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt',
       'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom',
       'hasGuestRoom', 'price'],
      dtype='object')


=== Features/Explanatory Variables ===

Continuous features: ['id', 'squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors', 'cityCode', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt', 'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom', 'hasGuestRoom', 'price']
Categorical features: []

 --- Cardinality of Categorical Features ---


=== Skewness ===

floors               85.12233
squareMeters         79.25331
made                 66.93441
basement              3.33664
attic                 2.80996
garage                1.37876
cityCode              0.24580
hasPool               0.18986
hasStormProtector     0.16049
hasStorageRoom        0.15374
price                 0.13140
isNewBuilt            0.12838
numberOfRooms         0.11508
hasYard               0.09656
id                    0.00000
numPrevOwners        -0.07711
cityPartRange        -0.09306
hasGuestRoom         -0.11777
dtype: float64
CPU times: user 56.6 ms, sys: 34 ms, total: 90.7 ms
Wall 

In [8]:
train.head()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8


In [9]:
original = pd.read_csv("../input/paris-housing-price-prediction/ParisHousing.csv")

original.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0


In [10]:
train['origin']    = 0
test['origin']     = 0
original['origin'] = 1
combined = pd.concat([train, original], ignore_index=True)
train = combined

In [11]:
combined.head()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price,origin
0,0.0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2,0
1,1.0,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0,0
2,2.0,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1,0
3,3.0,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2,0
4,4.0,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8,0


In [12]:
summary_statistics(train.drop(columns=[ID], axis=1), enhanced=True)

,count,mean,std,min,25%,50%,75%,max,var,skew,kurt
squareMeters,32730.00,47589.55,44252.70,89.00,21581.00,46132.00,72545.00,6071330.00,1958301742.77,77.09,10489.20
numberOfRooms,32730.00,48.89,28.42,1.00,25.00,48.00,75.00,100.00,807.94,0.09,-1.16
hasYard,32730.00,0.49,0.50,0.00,0.00,0.00,1.00,1.00,0.25,0.06,-2.00
hasPool,32730.00,0.47,0.50,0.00,0.00,0.00,1.00,1.00,0.25,0.14,-1.98
floors,32730.00,48.21,42.92,1.00,25.00,46.00,72.00,6000.00,1842.06,81.51,11298.59
cityCode,32730.00,50078.47,29704.41,3.00,23446.00,50452.00,76229.00,491100.00,882351901.11,0.17,1.43
cityPartRange,32730.00,5.56,2.78,1.00,3.00,6.00,8.00,10.00,7.73,-0.06,-1.17
numPrevOwners,32730.00,5.59,2.76,1.00,3.00,6.00,8.00,10.00,7.61,-0.05,-1.14
made,32730.00,2007.24,99.16,1990.00,2000.00,2006.00,2014.00,10000.00,9833.46,80.00,6446.00
isNewBuilt,32730.00,0.48,0.50,0.00,0.00,0.00,1.00,1.00,0.25,0.09,-1.99


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Feature Engineering</h1>
</div>

## Categorical/Numerical Variables

In [13]:
train.drop(['cityCode'], axis=1, inplace=True)
test.drop(['cityCode'], axis=1, inplace=True)


In [14]:
cont_features, cat_features = feature_distribution_types(train, display=True)
show_cardinality(train, cat_features)

cont_features.remove(TARGET)
cont_features.remove(ID)
FEATURES = cont_features + cat_features
FEATURES

Continuous Features=['id', 'squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt', 'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom', 'hasGuestRoom', 'price', 'origin']

Categorical Features=[]
=== Cardinality ===
Series([], dtype: float64)


['squareMeters',
 'numberOfRooms',
 'hasYard',
 'hasPool',
 'floors',
 'cityPartRange',
 'numPrevOwners',
 'made',
 'isNewBuilt',
 'hasStormProtector',
 'basement',
 'attic',
 'garage',
 'hasStorageRoom',
 'hasGuestRoom',
 'origin']

In [15]:
excluded_features = [TARGET, ID, "fold"]

In [16]:
cont_features, cat_features = feature_distribution_types(train, display=True)
show_cardinality(train, cat_features)

cont_features = [feature for feature in cont_features if feature not in excluded_features]
cat_features = [feature for feature in cat_features if feature not in excluded_features]

FEATURES = cont_features + cat_features
FEATURES

Continuous Features=['id', 'squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt', 'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom', 'hasGuestRoom', 'price', 'origin']

Categorical Features=[]
=== Cardinality ===
Series([], dtype: float64)


['squareMeters',
 'numberOfRooms',
 'hasYard',
 'hasPool',
 'floors',
 'cityPartRange',
 'numPrevOwners',
 'made',
 'isNewBuilt',
 'hasStormProtector',
 'basement',
 'attic',
 'garage',
 'hasStorageRoom',
 'hasGuestRoom',
 'origin']

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Train Models with Cross Validation</h1>
</div>

In [17]:
train = create_folds(train, Config.N_FOLDS)
# train = create_strat_folds(train, TARGET, Config.N_FOLDS)

n_folds=10, seed=42


In [18]:
all_cv_scores = pd.DataFrame(
    {
        "Model": pd.Series(dtype="str"),
        "Score": pd.Series(dtype="float"),
        "StdDev": pd.Series(dtype="float"),
        "RunTime": pd.Series(dtype="float"),
    }
)

oof = train[[ID, TARGET, "fold"]].copy().reset_index(drop=True).copy()
oof.set_index(ID, inplace=True)
oof.head()

,price,fold
id,,
0.0,3436795.2,4
1.0,9519958.0,7
2.0,9276448.1,6
3.0,9725732.2,2
4.0,6181908.8,2


In [19]:
def show_tree_model_fi(model, features:List[str]) -> None:
    print("\n=== Model Feature Importance ===")
    for i in model.feature_importances_.argsort()[::-1]:
        print(features[i], model.feature_importances_[i]/model.feature_importances_.sum())

def save_oof_predictions(model_name:str, final_valid_predictions, oof:pd.DataFrame) -> pd.DataFrame:
    final_valid_predictions_df = process_valid_predictions(
        final_valid_predictions, ID, model_name
    )
    display(final_valid_predictions_df.head())
    oof[f"pred_{model_name}"] = final_valid_predictions_df[f"pred_{model_name}"]

    return oof

def save_test_predictions(model_name:str, final_test_predictions, submission_df:pd.DataFrame, result_field:str=TARGET) -> None:
    result = merge_test_predictions(final_test_predictions, Config.calc_probability)
    # result[:20]
    submission_df[f"target_{model_name}"] = result #.astype(int)
    #     submission_df.head(10)
    ss = submission_df[[ID, f"target_{model_name}"]].copy().reset_index(drop=True)
    ss.rename(columns={f"target_{model_name}": result_field}, inplace=True)
    ss.to_csv(
        f"submission_{model_name}.csv", index=False
    )  # Can submit the individual model
    print("=== Target Value Counts ===")
#     display(ss[TARGET].value_counts())
    ss.head(10)

def process_valid_predictions(final_valid_predictions, train_id, model_name:str) -> pd.DataFrame:
    model = f"pred_{model_name}"
    final_valid_predictions_df = pd.DataFrame.from_dict(
        final_valid_predictions, orient="index"
    ).reset_index()
    final_valid_predictions_df.columns = [train_id, model]
    final_valid_predictions_df.set_index(train_id, inplace=True)
    final_valid_predictions_df.sort_index(inplace=True)
    final_valid_predictions_df.to_csv(f"train_pred_{model_name}.csv", index=True)

    return final_valid_predictions_df

def add_score(score_df:pd.DataFrame, model_name:str, score:float, std:float):
    dict1 = {"Model": model_name, "Score": cv_score, "StdDev": std_dev}
    score_df = score_df.append(dict1, ignore_index=True)
    return score_df

In [20]:
def train_cv_model(
    df:pd.DataFrame,
    test:pd.DataFrame,
    get_model_fn,
    FEATURES:List[str],
    TARGET:str,
    calc_probability:bool,
    rowid,
    params,
    n_folds:int=5,
    seed:int=42,
):

    final_test_predictions = []
    final_valid_predictions = {}
    fold_scores = []  # Scores of Validation Set
    feature_importance_lst = []

    test = test[FEATURES].copy()

    for fold in range(n_folds):
        print(10 * "=", f"Fold {fold+1}/{n_folds}", 10 * "=")

        start_time = time.time()

        xtrain = df[df.fold != fold].reset_index(
            drop=True
        )  # Everything not in validation fold
        xvalid = df[df.fold == fold].reset_index(drop=True)
        xtest = test.copy()

        valid_ids = xvalid.id.values.tolist()  # Id's of everything in validation fold

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[FEATURES]
        xvalid = xvalid[FEATURES]

        scaler = preprocessing.StandardScaler()
#         scaler = preprocessing.MinMaxScaler()
        xtrain = scaler.fit(xtrain).transform(xtrain)
        xvalid = scaler.transform(xvalid)
        xtest = scaler.transform(xtest)

        model = get_model_fn # ()

        model.fit(
            xtrain,
            ytrain,
        )
        if calc_probability:
            preds_valid = model.predict_proba(xvalid)[:, 1]
            test_preds = model.predict_proba(xtest)[:, 1]
        else:
            preds_valid = model.predict(xvalid)
            test_preds = model.predict(xtest)

        preds_valid_class = model.predict(xvalid)
        
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))

#         fold_score = metrics.accuracy_score(yvalid, preds_valid_class)  # Validation Set Score
        fold_score = metrics.mean_absolute_error(
            yvalid, preds_valid
        ) 
#         fold_score = metrics.roc_auc_score(yvalid.values, preds_valid)  # Validation Set Score
#         show_classification_scores(yvalid.values, preds_valid_class)

#         fold_score = metrics.roc_auc_score(yvalid, preds_valid)  # Validation Set Score
#         fold_score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        fold_scores.append(fold_score)
        #         importance_list.append(model.coef_.ravel())

        fi = []
        # Feature importance
#         fi = pd.DataFrame(
#             index=FEATURES,
#             data=model.coef_.ravel(),
#             columns=[f"{fold}_importance"],
#         )
        
        feature_importance_lst.append(fi)

        run_time = time.time() - start_time

        print(f"fold: {fold+1}, Score: {fold_score}, Run Time: {run_time:.2f}")

    return (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    )


def train_xgb_model(
    df:pd.DataFrame,
    test:pd.DataFrame,
    get_model_fn,
    FEATURES:List[str],
    TARGET:str,
    calc_probability:bool,
    rowid:str,
    params,
    n_folds:int=5,
    seed:int=42,
):

    print(params)
    final_test_predictions = []
    final_valid_predictions = {}
    fold_scores = []  # Scores of Validation Set
    feature_importance_lst = []

    test = test[FEATURES].copy()

    for fold in range(n_folds):
        print(10 * "=", f"Fold {fold+1}/{n_folds}", 10 * "=")

        start_time = time.time()

        xtrain = df[df.fold != fold].reset_index(
            drop=True
        )  # Everything not in validation fold
        xvalid = df[df.fold == fold].reset_index(drop=True)
        xtest = test.copy()

        valid_ids = xvalid.id.values.tolist()  # Id's of everything in validation fold

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[FEATURES]
        xvalid = xvalid[FEATURES]

        model = get_model_fn # (params)

        model.fit(
            xtrain,
            ytrain,
            eval_set=[(xvalid, yvalid)],
            #             eval_metric="acc",  # auc
            verbose=0,
            #             early_stopping_rounds=3000,
            #             callbacks=[
            #                 xgb.log_evaluation(0),
            #                 xgb.early_stopping(500, False, True),
            #             ],
        )

        if calc_probability:
            preds_valid = model.predict_proba(xvalid)[:, 1]
            test_preds = model.predict_proba(xtest)[:, 1]
        else:
            preds_valid = model.predict(xvalid)
            test_preds = model.predict(xtest)

        preds_valid_class = model.predict(xvalid)
        
        final_test_predictions.append(test_preds)
        if Config.debug:
            print(f"GT Type: {type(yvalid.values)}")
            print(f"Preds Type: {type(preds_valid_class)}")
            print(f"         GT:{yvalid.values[:20]}")
            print(f"Preds Class:{preds_valid_class[:20]}")
            print(f"Preds Prob:{preds_valid[:20]}")
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid_class)))

#         fold_score = metrics.cohen_kappa_score(yvalid,  preds_valid_class, weights = "quadratic")
#         fold_score = metrics.roc_auc_score(yvalid.values, preds_valid)  # Validation Set Score
#         show_classification_scores(yvalid.values, preds_valid_class)
        fold_score = metrics.mean_absolute_error(
            yvalid, preds_valid
        )  # Validation Set Score
#         fold_score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        fold_scores.append(fold_score)

        # Feature importance
        fi = pd.DataFrame(
            index=FEATURES,
            data=model.feature_importances_,
            columns=[f"{fold}_importance"],
        )
        feature_importance_lst.append(fi)

        run_time = time.time() - start_time

        print(f"fold: {fold+1}, Score: {fold_score}, Run Time: {run_time:.2f}")

    return (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    )        

In [21]:
def run_linear_model(model_dict, model_name:str, features:List[str], oof:pd.DataFrame) -> (float, float, pd.DataFrame):
    (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    ) = train_cv_model(
        train,
        test,
        model_dict[model_name],
        features,
        TARGET,
        False, #Config.calc_probability,
        ID,
        {},
        Config.N_FOLDS,
        Config.seed,
    )

    cv_score, std_dev = show_fold_scores(fold_scores)

    oof = save_oof_predictions(model_name, final_valid_predictions, oof)
    oof.head()
    save_test_predictions(model_name, final_test_predictions, sample_submission, TARGET)

    return cv_score, std_dev, oof


def run_tree_model(model_dict, model_name:str, features:List[str], params, oof:pd.DataFrame) -> (float, float, pd.DataFrame):
    (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    ) = train_xgb_model(
        train,
        test,
        model_dict[model_name],
        features,
        TARGET,
        Config.calc_probability,
        ID,
        params,
        Config.N_FOLDS,
        Config.seed,
    )

    cv_score, std_dev = show_fold_scores(fold_scores)
    show_tree_model_fi(model, features)

    oof = save_oof_predictions(model_name, final_valid_predictions, oof)
    oof.head()
    save_test_predictions(model_name, final_test_predictions, sample_submission, TARGET)

    return cv_score, std_dev, oof

In [22]:
%%time

def run_models4features(model_dict, model_lst:List[str], target:str, feature_lst:List[str], all_cv_scores:pd.DataFrame, linear_models:bool=True) -> pd.DataFrame:

    oof = train[[ID, target, "fold"]].copy().reset_index(drop=True).copy()
    oof.set_index(ID, inplace=True)

    for idx, m in enumerate(model_lst):
        model = model_lst[idx]
        start_time = time.time()

        print(f"Model={model}")

        params = {}
        if linear_models:
                cv_score, std_dev, oof = run_linear_model(model_dict, model, feature_lst, oof)

        else:
            cv_score, std_dev, oof = run_tree_model(model_dict, model, feature_lst, params, oof)

        run_time = time.time() - start_time

        score_dict = {"Model": model, "Score": cv_score, "StdDev": std_dev, "RunTime": run_time}
        all_cv_scores = all_cv_scores.append(score_dict, ignore_index=True)
        print(f"Model Run Time: {run_time:.2f}")

    return all_cv_scores




CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 14.5 µs


In [23]:
lgbm_params = {'n_estimators': Config.N_ESTIMATORS,
                 'num_rounds': 404,
                 'learning_rate': 0.19,
                 'num_leaves': 17,
                 'max_depth': 8,
                 'min_data_in_leaf': 36,
                 'lambda_l1': 0.96,
                 'lambda_l2': 0.01,
                 'min_gain_to_split': 11.32,
                 'bagging_fraction': 0.6,
                 'feature_fraction': 0.9}


lgbm_params = gpu_ify_lgbm(lgbm_params)
# if Config.gpu:
#     lgbm_params["device"] = "gpu"
#     lgbm_params["boosting_type"] = "gbdt"
#     lgbm_params["gpu_platform_id"] = 0
#     lgbm_params["gpu_device_id"] = 0

In [24]:
xgb_params = {
    "n_estimators": Config.N_ESTIMATORS,  # 10_000,
    "max_depth": 10,  # 10
    "objective": "reg:squarederror",
    #     "enable_categorical": True,  # Only works with gpu_hist
    #     "eval_metric": "mae",
    #     "metric": "mae",
    #     "enable_categorical": True,
    "n_jobs": 8,  # 4
    "seed": Config.seed,
    "tree_method": "hist",
    #         "gpu_id": 0,
    "subsample": 0.9,  # 0.7
    "colsample_bytree": 0.7,
    "use_label_encoder": False,
    "learning_rate": 0.05,  # 0.01
}

if Config.gpu:
    xgb_params["tree_method"] = "gpu_hist"
else:
    xgb_params["tree_method"] = "hist"

In [25]:
cb_params = {
    #     "learning_rate": 0.3277295792305584,
    "learning_rate": 0.05,
    "l2_leaf_reg": 3.1572972266001518,
    "bagging_temperature": 0.6799604234141348,
    "random_strength": 1.99590400593318,
    "depth": 10,
    "min_data_in_leaf": 93,
    # "iterations": 100,  # 10000
    "n_estimators": Config.N_ESTIMATORS,  # 10000
    "use_best_model": True,
    #     "task_type": "GPU",
    "random_seed": Config.seed,
}

cb_params = gpu_ify_cb(cb_params)
# if Config.gpu:
#     cb_params["task_type"] = "GPU"

In [26]:
lgbm_params = {
    "n_estimators": Config.GPU_N_ESTIMATORS,
    'max_depth': 9,
    'learning_rate': 0.01,
    'min_data_in_leaf': 36, 
    'num_leaves': 100, 
    'feature_fraction': 0.8, 
    'bagging_fraction': 0.89, 
    'bagging_freq': 5, 
    'lambda_l2': 28,
    
    'seed': Config.seed,
    'objective': 'regression',
#     'boosting_type': 'gbdt',
#     'device': 'gpu', 
#     'gpu_platform_id': 0,
#     'gpu_device_id': 0,
    'n_jobs': -1,
    'metric': 'rmse',
    'verbose': -1
}

if Config.gpu:
    lgbm_params["device"] = "gpu"
    lgbm_params["boosting_type"] = "gbdt"
    lgbm_params["gpu_platform_id"] = 0
    lgbm_params["gpu_device_id"] = 0

In [27]:
model_estimator_dict = {
    "xgb2": xgb.XGBRegressor(**xgb_params),
#     "lgbm1": lgb.LGBMRegressor(**lgbm_params),

    "cat2": cb.CatBoostRegressor(**cb_params),

    "xgb1": xgb.XGBRegressor(),
    "lgbm1": lgb.LGBMRegressor(),
    "lgbm1": lgb.LGBMRegressor(),
    "lgbm2": lgb.LGBMRegressor(
        learning_rate=0.05,
        max_depth=15,
        num_leaves=11,
        feature_fraction=0.3,
        subsample=0.1,
        n_jobs=-1,
    ),
    "lgbm3": lgb.LGBMRegressor(**lgbm_params),

    "cat1": cb.CatBoostRegressor(),

    "lin_reg": linear_model.LinearRegression(),
    "lasso": linear_model.Lasso(),
    "ridge": linear_model.Ridge(max_iter=7000),
    "ridge_25": linear_model.Ridge(fit_intercept=True, solver='auto', alpha=0.25, max_iter=7000),
    "ridge_50": linear_model.Ridge(fit_intercept=True, solver='auto', alpha=0.5, max_iter=7000),

}

## Tree Models

In [28]:
%%time

model_lst = ["xgb1", "xgb2", "lgbm1", "lgbm2", "cat1", "cat2"]
# model_lst = ["lgbm1"]
# model_lst = = []
all_cv_scores = run_models4features(model_estimator_dict, model_lst, TARGET, FEATURES, all_cv_scores, linear_models=False)    

all_cv_scores.sort_values(by=["Score"], ascending=False)

Model=xgb1
{}
========== Fold 1/10 ==========
fold: 1, Score: 15341.331424217795, Run Time: 4.33
========== Fold 2/10 ==========
fold: 2, Score: 19589.669087082653, Run Time: 3.22
========== Fold 3/10 ==========
fold: 3, Score: 16141.766257818772, Run Time: 2.91
========== Fold 4/10 ==========
fold: 4, Score: 19495.708808337153, Run Time: 2.89
========== Fold 5/10 ==========
fold: 5, Score: 17442.182938650516, Run Time: 3.02
========== Fold 6/10 ==========
fold: 6, Score: 17250.63759055506, Run Time: 3.46
========== Fold 7/10 ==========
fold: 7, Score: 18907.2681235738, Run Time: 3.73
========== Fold 8/10 ==========
fold: 8, Score: 16969.379939383303, Run Time: 3.02
========== Fold 9/10 ==========
fold: 9, Score: 20111.190821684984, Run Time: 3.47
========== Fold 10/10 ==========
fold: 10, Score: 14293.863043032006, Run Time: 3.15
Scores -> Adjusted: 15703.62824664 , mean: 17554.29980343, std: 1850.67155680

=== Model Feature Importance ===
squareMeters 0.98601454
hasYard 0.004066093
m

,pred_xgb1
id,
0.0,3410296.0
1.0,9522324.0
2.0,9272929.0
3.0,9751212.0
4.0,6182113.5


Mode
=== Target Value Counts ===
Model Run Time: 33.72
Model=xgb2
{}
========== Fold 1/10 ==========
fold: 1, Score: 126662.60684464988, Run Time: 77.16
========== Fold 2/10 ==========
fold: 2, Score: 138828.31472138877, Run Time: 74.79
========== Fold 3/10 ==========
fold: 3, Score: 139967.84993137507, Run Time: 74.98
========== Fold 4/10 ==========
fold: 4, Score: 134914.31986514007, Run Time: 76.43
========== Fold 5/10 ==========
fold: 5, Score: 137302.8764447486, Run Time: 75.98
========== Fold 6/10 ==========
fold: 6, Score: 139167.32037020463, Run Time: 76.41
========== Fold 7/10 ==========
fold: 7, Score: 142043.10550586713, Run Time: 75.87
========== Fold 8/10 ==========
fold: 8, Score: 136273.8952843936, Run Time: 76.12
========== Fold 9/10 ==========
fold: 9, Score: 142235.7956032608, Run Time: 75.31
========== Fold 10/10 ==========
fold: 10, Score: 139834.8051048589, Run Time: 76.53
Scores -> Adjusted: 133429.68846485 , mean: 137723.08896759, std: 4293.40050274

=== Model Fe

,pred_xgb2
id,
0.0,3550867.5
1.0,9095681.0
2.0,9180069.0
3.0,9486013.0
4.0,6191376.0


Mode
=== Target Value Counts ===
Model Run Time: 761.39
Model=lgbm1
{}
========== Fold 1/10 ==========
fold: 1, Score: 21053.27732884381, Run Time: 0.66
========== Fold 2/10 ==========
fold: 2, Score: 26075.610050441857, Run Time: 0.67
========== Fold 3/10 ==========
fold: 3, Score: 22366.075328084124, Run Time: 0.70
========== Fold 4/10 ==========
fold: 4, Score: 23337.06703372781, Run Time: 0.72
========== Fold 5/10 ==========
fold: 5, Score: 25480.329196222934, Run Time: 0.75
========== Fold 6/10 ==========
fold: 6, Score: 22961.484316323724, Run Time: 1.19
========== Fold 7/10 ==========
fold: 7, Score: 24820.257311907204, Run Time: 0.85
========== Fold 8/10 ==========
fold: 8, Score: 22376.097888573062, Run Time: 0.85
========== Fold 9/10 ==========
fold: 9, Score: 27292.023697526234, Run Time: 0.90
========== Fold 10/10 ==========
fold: 10, Score: 22778.11331921407, Run Time: 0.94
Scores -> Adjusted: 21988.62056211 , mean: 23854.03354709, std: 1865.41298497

=== Model Feature Imp

...

,pred_cat1
id,
0.0,3.44499e+06
1.0,9.49296e+06
2.0,9.25729e+06
3.0,9.71944e+06
4.0,6.12016e+06


Mode
=== Target Value Counts ===
Model Run Time: 63.67
Model=cat2
{}
========== Fold 1/10 ==========
fold: 1, Score: 25309.22862968883, Run Time: 114.58
========== Fold 2/10 ==========
fold: 2, Score: 32969.41884260525, Run Time: 31.30
========== Fold 3/10 ==========
fold: 3, Score: 29514.97909551272, Run Time: 34.58
========== Fold 4/10 ==========
fold: 4, Score: 28578.4790888758, Run Time: 31.84
========== Fold 5/10 ==========
fold: 5, Score: 34042.83560687858, Run Time: 30.89
========== Fold 6/10 ==========
fold: 6, Score: 31027.702639688665, Run Time: 37.59
========== Fold 7/10 ==========
fold: 7, Score: 31607.904445775734, Run Time: 31.82
========== Fold 8/10 ==========
fold: 8, Score: 28499.28507000721, Run Time: 42.78
========== Fold 9/10 ==========
fold: 9, Score: 32461.451017503823, Run Time: 32.17
========== Fold 10/10 ==========
fold: 10, Score: 28197.213021917614, Run Time: 33.29
Scores -> Adjusted: 27688.34147034 , mean: 30220.84974585, std: 2532.50827551

=== Model Featur

,pred_cat2
id,
0.0,3.49284e+06
1.0,9.50787e+06
2.0,9.26140e+06
3.0,9.67003e+06
4.0,6.18562e+06


Mode
=== Target Value Counts ===
Model Run Time: 421.52
CPU times: user 23min 44s, sys: 1min 27s, total: 25min 11s
Wall time: 21min 35s


,Model,Score,StdDev,RunTime
3,lgbm2,896471.40730,9133.25418,6.36757
1,xgb2,137723.08897,4293.40050,761.38934
4,cat1,33472.36758,6824.65819,63.66628
5,cat2,30220.84975,2532.50828,421.52290
2,lgbm1,23854.03355,1865.41298,8.79798
0,xgb1,17554.29980,1850.67156,33.72312


## Linear Models

In [29]:
model_lst = ["lin_reg", "lasso", "ridge", "ridge_25", "ridge_50"]
model_lst = ["lasso", "ridge",  "ridge_50"]
# model_lst = []
# all_cv_scores = run_models4features(model_lst, TARGET, FEATURES, all_cv_scores, linear_models=True)    
all_cv_scores = run_models4features(model_estimator_dict, model_lst, TARGET, FEATURES, all_cv_scores, linear_models=True)    

all_cv_scores.head()

Model=lasso
========== Fold 1/10 ==========
fold: 1, Score: 1484398.1407285521, Run Time: 0.09
========== Fold 2/10 ==========
fold: 2, Score: 195969.9041634135, Run Time: 0.20
========== Fold 3/10 ==========
fold: 3, Score: 1482389.440692571, Run Time: 0.21
========== Fold 4/10 ==========
fold: 4, Score: 1489882.1844967173, Run Time: 0.18
========== Fold 5/10 ==========
fold: 5, Score: 1473213.0797100973, Run Time: 0.21
========== Fold 6/10 ==========
fold: 6, Score: 1482455.6031536742, Run Time: 0.24
========== Fold 7/10 ==========
fold: 7, Score: 1451686.951608927, Run Time: 0.33
========== Fold 8/10 ==========
fold: 8, Score: 1497041.3950264666, Run Time: 0.31
========== Fold 9/10 ==========
fold: 9, Score: 1480775.1781013324, Run Time: 0.36
========== Fold 10/10 ==========
fold: 10, Score: 1506569.8045058148, Run Time: 0.41
Scores -> Adjusted: 968038.25269202 , mean: 1354438.16821876, std: 386399.91552674


,pred_lasso
id,
0.0,4.29894e+06
1.0,6.99111e+06
2.0,6.67209e+06
3.0,7.11385e+06
4.0,5.18365e+06


Mode
=== Target Value Counts ===
Model Run Time: 3.07
Model=ridge
========== Fold 1/10 ==========
fold: 1, Score: 1484432.1700286455, Run Time: 0.09
========== Fold 2/10 ==========
fold: 2, Score: 196015.58739982042, Run Time: 0.15
========== Fold 3/10 ==========
fold: 3, Score: 1482423.2544933106, Run Time: 0.18
========== Fold 4/10 ==========
fold: 4, Score: 1489915.8412414195, Run Time: 0.20
========== Fold 5/10 ==========
fold: 5, Score: 1473246.7901216762, Run Time: 0.22
========== Fold 6/10 ==========
fold: 6, Score: 1482489.230168893, Run Time: 0.25
========== Fold 7/10 ==========
fold: 7, Score: 1451720.3869964648, Run Time: 0.33
========== Fold 8/10 ==========
fold: 8, Score: 1497075.3645273896, Run Time: 0.32
========== Fold 9/10 ==========
fold: 9, Score: 1480809.057370939, Run Time: 0.37
========== Fold 10/10 ==========
fold: 10, Score: 1506604.2389542812, Run Time: 0.42
Scores -> Adjusted: 968076.82001200 , mean: 1354473.19213028, std: 386396.37211828


,pred_ridge
id,
0.0,4.29896e+06
1.0,6.99105e+06
2.0,6.67202e+06
3.0,7.11378e+06
4.0,5.18363e+06


Mode
=== Target Value Counts ===
Model Run Time: 3.12
Model=ridge_50
========== Fold 1/10 ==========
fold: 1, Score: 1484414.9195048942, Run Time: 0.07
========== Fold 2/10 ==========
fold: 2, Score: 195993.2725089869, Run Time: 0.14
========== Fold 3/10 ==========
fold: 3, Score: 1482406.1414678078, Run Time: 0.18
========== Fold 4/10 ==========
fold: 4, Score: 1489898.7723160542, Run Time: 0.20
========== Fold 5/10 ==========
fold: 5, Score: 1473229.6802832314, Run Time: 0.23
========== Fold 6/10 ==========
fold: 6, Score: 1482472.1785917059, Run Time: 0.26
========== Fold 7/10 ==========
fold: 7, Score: 1451703.3839907905, Run Time: 0.34
========== Fold 8/10 ==========
fold: 8, Score: 1497058.1620561467, Run Time: 0.32
========== Fold 9/10 ==========
fold: 9, Score: 1480791.9207652688, Run Time: 0.40
========== Fold 10/10 ==========
fold: 10, Score: 1506586.7915272932, Run Time: 0.42
Scores -> Adjusted: 968057.60602209 , mean: 1354455.52230122, std: 386397.91627913


,pred_ridge_50
id,
0.0,4.29895e+06
1.0,6.99109e+06
2.0,6.67205e+06
3.0,7.11382e+06
4.0,5.18363e+06


Mode
=== Target Value Counts ===
Model Run Time: 3.17


,Model,Score,StdDev,RunTime
0,xgb1,17554.29980,1850.67156,33.72312
1,xgb2,137723.08897,4293.40050,761.38934
2,lgbm1,23854.03355,1865.41298,8.79798
3,lgbm2,896471.40730,9133.25418,6.36757
4,cat1,33472.36758,6824.65819,63.66628


In [30]:
sample_submission.head(20)

,id,price,target_xgb1,target_xgb2,target_lgbm1,target_lgbm2,target_cat1,target_cat2,target_lasso,target_ridge,target_ridge_50
0,22730,4.63446e+06,4.74203e+06,4.57876e+06,4.74967e+06,4.31745e+06,4.75730e+06,4.70683e+06,4.76137e+06,4.76137e+06,4.76137e+06
1,22731,4.63446e+06,6.21005e+06,6.03375e+06,6.19309e+06,5.57261e+06,6.12239e+06,6.17622e+06,5.31879e+06,5.31877e+06,5.31878e+06
2,22732,4.63446e+06,9.05729e+06,9.01673e+06,9.03814e+06,7.32494e+06,9.04181e+06,9.05015e+06,6.62846e+06,6.62840e+06,6.62843e+06
3,22733,4.63446e+06,1.55696e+06,1.40587e+06,1.61705e+06,2.82738e+06,1.59827e+06,1.62248e+06,1.64826e+06,1.64839e+06,1.64833e+06
4,22734,4.63446e+06,6.75592e+06,6.63726e+06,6.73953e+06,6.05746e+06,6.64678e+06,6.71577e+06,5.41997e+06,5.41994e+06,5.41995e+06
5,22735,4.63446e+06,1.08373e+05,4.57040e+04,1.05743e+05,2.20130e+06,1.02498e+05,9.79815e+04,1.13199e+05,1.13367e+05,1.13284e+05
6,22736,4.63446e+06,9.90177e+06,9.36084e+06,9.87499e+06,7.40698e+06,9.87278e+06,9.81258e+06,6.59282e+06,6.59275e+06,6.59278e+06
7,22737,4.63446e+06,5.22909e+06,5.21865e+06,5.23145e+06,5.05902e+06,5.14971e+06,5.23049e+06,4.75583e+06,4.75583e+06,4.75583e+06
8,22738,4.63446e+06,5.56040e+06,5.39159e+06,5.56700e+06,5.13154e+06,5.58051e+06,5.54687e+06,4.87243e+06,4.87242e+06,4.87242e+06
9,22739,4.63446e+06,9.82945e+06,9.53208e+06,9.69814e+06,7.46131e+06,9.76865e+06,9.76025e+06,6.41732e+06,6.41725e+06,6.41728e+06


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Blend Models</h1>
</div>

In [31]:
all_blend_scores = pd.DataFrame(
    {
        "Model": pd.Series(dtype="str"),
        "Score": pd.Series(dtype="float"),
        "StdDev": pd.Series(dtype="float"),
    }
)

In [32]:
sample_submission[TARGET] = (
#     (sample_submission["target_xgb_bp"] * 2 )
#     + (sample_submission["target_lgbm_bp"]  )
    (sample_submission["target_xgb1"] * 2 )
    + (sample_submission["target_lgbm1"])
#     + (sample_submission["target_lgbm2"])    
#     + (sample_submission["target_lgbm2"])
    + (sample_submission["target_cat1"] )
    + (sample_submission["target_cat2"] )    
#     + (sample_submission["target_cat_bp"] )
#     + (sample_submission["target_svc"] )
#     + (sample_submission["target_log_reg3"] )
#     + (sample_submission["target_cat2"] )
)/5

sample_submission[TARGET] = sample_submission[TARGET] #.astype(int)

In [33]:
sample_submission[[ID, TARGET]].to_csv("submission_wt_avg.csv", index=False)
sample_submission[[ID, TARGET]].tail(8)

,id,price
15146,37876,6.82803e+06
15147,37877,2.14887e+06
15148,37878,3.07002e+06
15149,37879,8.62241e+06
15150,37880,4.52170e+06
15151,37881,7.33431e+06
15152,37882,7.74103e+06
15153,37883,1.76910e+06


In [34]:
all_cv_scores.sort_values(by=["Score"], ascending=False)

,Model,Score,StdDev,RunTime
7,ridge,1.35447e+06,386396.37212,3.11755
8,ridge_50,1.35446e+06,386397.91628,3.16890
6,lasso,1.35444e+06,386399.91553,3.07485
3,lgbm2,8.96471e+05,9133.25418,6.36757
1,xgb2,1.37723e+05,4293.40050,761.38934
4,cat1,3.34724e+04,6824.65819,63.66628
5,cat2,3.02208e+04,2532.50828,421.52290
2,lgbm1,2.38540e+04,1865.41298,8.79798
0,xgb1,1.75543e+04,1850.67156,33.72312


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Level 1 Stack Models</h1>
</div>

In [35]:
## TODO: Generate these dictionaries from model names

train_oof_dict = {
    "train_pred_cat1": "train_pred_cat1.csv",
    "train_pred_cat2": "train_pred_cat2.csv",
    "train_pred_lgbm1": "train_pred_lgbm1.csv",    
    "train_pred_lgbm2": "train_pred_lgbm2.csv",    
    "train_pred_xgb1": "train_pred_xgb1.csv"
}

test_pred_dict = {
    "submission_cat1": "submission_cat1.csv",
    "submission_cat2": "submission_cat2.csv",
    "submission_lgbm1": "submission_lgbm1.csv",
    "submission_lgbm2": "submission_lgbm2.csv",
    "submission_xgb1": "submission_xgb1.csv",
}

In [36]:
def blend_results(train_oof_dict, test_pred_dict):
    oof_df = pd.DataFrame()
    test_preds_df = pd.DataFrame()

    for name, train_oof_fname in train_oof_dict.items():
        fname = "../working/" + train_oof_fname
        print(f"Processing {name}, {train_oof_fname}")
        df = pd.read_csv(fname)
        print(df.head())
#         print(df.iloc[:,1])
        preds = pd.Series(df.iloc[:,1], name=name)
#         print(preds[:5])
        oof_df = pd.concat([oof_df, preds], axis=1)
    #     oof_df = pd.concat([oof_df, pd.Series(np.load(TRAIN_PATH / train_oof), name=name)], axis=1)

    for name, test_pred_fname in test_pred_dict.items():
        fname = "../working/" + test_pred_fname
        print(f"{name}, {test_pred_fname}")
        df = pd.read_csv(fname)
        print(df.head())
        preds = pd.Series(df.iloc[:,1], name=name)
        test_preds_df = pd.concat([test_preds_df, preds], axis=1)

    print("=== oof ===")
    print(oof_df.head())
    print("=== test_preds ===")
    print(test_preds_df.head())
    return oof_df, test_preds_df
    
# (oof_df, preds_df) = blend_results(train_oof_dict, test_pred_dict)    

In [37]:
def load_oof_results(train_oof_dict, test_pred_dict):
    oof_df = pd.DataFrame()
    test_preds_df = pd.DataFrame()

    for name, train_oof_fname in train_oof_dict.items():
        fname = "../working/" + train_oof_fname
        print(f"Processing {name}, {train_oof_fname}")
        df = pd.read_csv(fname)
        print(df.head())
#         print(df.iloc[:,1])
        preds = pd.Series(df.iloc[:,1], name=name)
#         print(preds[:5])
        oof_df = pd.concat([oof_df, preds], axis=1)
    #     oof_df = pd.concat([oof_df, pd.Series(np.load(TRAIN_PATH / train_oof), name=name)], axis=1)

    for name, test_pred_fname in test_pred_dict.items():
        fname = "../working/" + test_pred_fname
        print(f"{name}, {test_pred_fname}")
        df = pd.read_csv(fname)
        print(df.head())
        preds = pd.Series(df.iloc[:,1], name=name)
        test_preds_df = pd.concat([test_preds_df, preds], axis=1)

    print("=== oof ===")
    print(oof_df.head())
    print("=== test_preds ===")
    print(test_preds_df.head())
    return oof_df, test_preds_df
    
(oof_df, preds_df) = load_oof_results(train_oof_dict, test_pred_dict) 

Processing train_pred_cat1, train_pred_cat1.csv
    id    pred_cat1
0  0.0  3.44499e+06
1  1.0  9.49296e+06
2  2.0  9.25729e+06
3  3.0  9.71944e+06
4  4.0  6.12016e+06
Processing train_pred_cat2, train_pred_cat2.csv
    id    pred_cat2
0  0.0  3.49284e+06
1  1.0  9.50787e+06
2  2.0  9.26140e+06
3  3.0  9.67003e+06
4  4.0  6.18562e+06
Processing train_pred_lgbm1, train_pred_lgbm1.csv
    id   pred_lgbm1
0  0.0  3.45914e+06
1  1.0  9.54216e+06
2  2.0  9.26341e+06
3  3.0  9.74807e+06
4  4.0  6.19148e+06
Processing train_pred_lgbm2, train_pred_lgbm2.csv
    id   pred_lgbm2
0  0.0  3.80021e+06
1  1.0  7.49571e+06
2  2.0  7.75381e+06
3  3.0  7.89862e+06
4  4.0  5.61409e+06
Processing train_pred_xgb1, train_pred_xgb1.csv
    id  pred_xgb1
0  0.0  3410296.0
1  1.0  9522324.0
2  2.0  9272929.0
3  3.0  9751212.0
4  4.0  6182113.5
submission_cat1, submission_cat1.csv
      id        price
0  22730  4.75730e+06
1  22731  6.12239e+06
2  22732  9.04181e+06
3  22733  1.59827e+06
4  22734  6.64678e+06

In [38]:
oof_df.head()

,train_pred_cat1,train_pred_cat2,train_pred_lgbm1,train_pred_lgbm2,train_pred_xgb1
0,3.44499e+06,3.49284e+06,3.45914e+06,3.80021e+06,3410296.0
1,9.49296e+06,9.50787e+06,9.54216e+06,7.49571e+06,9522324.0
2,9.25729e+06,9.26140e+06,9.26341e+06,7.75381e+06,9272929.0
3,9.71944e+06,9.67003e+06,9.74807e+06,7.89862e+06,9751212.0
4,6.12016e+06,6.18562e+06,6.19148e+06,5.61409e+06,6182113.5


In [39]:
preds_df.head()

,submission_cat1,submission_cat2,submission_lgbm1,submission_lgbm2,submission_xgb1
0,4.75730e+06,4.70683e+06,4.74967e+06,4.31745e+06,4742034.5
1,6.12239e+06,6.17622e+06,6.19309e+06,5.57261e+06,6210045.5
2,9.04181e+06,9.05015e+06,9.03814e+06,7.32494e+06,9057293.0
3,1.59827e+06,1.62248e+06,1.61705e+06,2.82738e+06,1556964.1
4,6.64678e+06,6.71577e+06,6.73953e+06,6.05746e+06,6755925.0


In [40]:
type(preds_df)

pandas.core.frame.DataFrame

In [41]:
def run_lr(useful_features:List[str], TARGET:str, train_df:pd.DataFrame, test_df:pd.DataFrame) -> (List[float],List[float]):
    final_predictions = []
    scores = []

    kfold = model_selection.KFold(n_splits=Config.N_FOLDS, shuffle=True, random_state=Config.seed)

    for fold, (train_idx, valid_idx) in enumerate(kfold.split(train_df)):
        xtrain = train_df.iloc[train_idx].reset_index(drop=True)
        xvalid = train_df.iloc[valid_idx].reset_index(drop=True)

        xtest = test_df[useful_features].copy()

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[useful_features]
        xvalid = xvalid[useful_features]

#         model = LogisticRegression()
        model = linear_model.LinearRegression()
        # Smaller C means more regularization; default=1.0
        # 2947.0517025518097
#         model = LogisticRegression(max_iter=500, C=2947.0517025518097, penalty='l2',solver='newton-cg')
#         model = LogisticRegression(C = 2947.0517025518097,
#                         max_iter = 500,
#                         penalty = 'l2',
#                         solver = 'liblinear')
        model.fit(xtrain, ytrain)

        preds_valid = model.predict_proba(xvalid)[:,-1]
        test_preds = model.predict_proba(xtest)[:,-1]

        final_predictions.append(test_preds)
#         score = metrics.roc_auc_score(yvalid, preds_valid)
        score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        print(f"Fold={fold}, Score={score}")
        scores.append(score)
    return scores, final_predictions


In [42]:
# useful_features = ["pred_lda", "pred_gbc","pred_gbc2", "pred_cat_bp", "pred_cat1", "pred_lgbm1", "pred_lgbm2", "pred_lgbm_bp", "pred_xgb1", "pred_xgb_bp"]
useful_features = [ "train_pred_cat1", "train_pred_cat2", "train_pred_lgbm1", "train_pred_lgbm2", "train_pred_xgb1"]

In [43]:
oof_df[useful_features].head()

,train_pred_cat1,train_pred_cat2,train_pred_lgbm1,train_pred_lgbm2,train_pred_xgb1
0,3.44499e+06,3.49284e+06,3.45914e+06,3.80021e+06,3410296.0
1,9.49296e+06,9.50787e+06,9.54216e+06,7.49571e+06,9522324.0
2,9.25729e+06,9.26140e+06,9.26341e+06,7.75381e+06,9272929.0
3,9.71944e+06,9.67003e+06,9.74807e+06,7.89862e+06,9751212.0
4,6.12016e+06,6.18562e+06,6.19148e+06,5.61409e+06,6182113.5


In [44]:
# preds_df[useful_features].head()

In [45]:
# fold_scores, final_predictions = run_lr(useful_features, TARGET, oof_df, preds_df)
# test_preds = np.mean(np.column_stack(final_predictions), axis=1)
# cv_score, std_dev = show_fold_scores(fold_scores)
# create_submission("level1_lr", TARGET, test_preds)

In [46]:
pd.options.display.max_colwidth = 100
pd.set_option("display.max_rows", 999)
pd.set_option("display.precision", 5)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_colwidth



100

In [47]:
all_cv_scores.sort_values(by=["Score"], ascending=False)

,Model,Score,StdDev,RunTime
7,ridge,1354473.19,386396.37,3.12
8,ridge_50,1354455.52,386397.92,3.17
6,lasso,1354438.17,386399.92,3.07
3,lgbm2,896471.41,9133.25,6.37
1,xgb2,137723.09,4293.40,761.39
4,cat1,33472.37,6824.66,63.67
5,cat2,30220.85,2532.51,421.52
2,lgbm1,23854.03,1865.41,8.80
0,xgb1,17554.30,1850.67,33.72
